In [67]:
import numpy as np

class HMM():
    def __init__(self, transition, mu,sigma, obs,T,pi):
        self.transition = transition #shape 4x4
        self.state=[0,1,2,3] #a modifier 
        self.pi=pi
        self.mu = mu
        self.sigma = sigma
        self.obs=obs
        self.T=T
        self.q=np.zeros(T)

    
    def  multivariate_gaussian(self, xi):
        """ Computes the vector of the probabilities p(u_t | q_t=i )"""
        probs = np.zeros(4)
        mu = self.mu
        sigmas = self.sigma
        inv_sigmas = [np.linalg.inv(sigmas[j]) for j in range(4)]
        det_sigmas = [np.linalg.det(sigmas[j]) for j in range(4)]
        for j in range(4):
            x_m=(xi-mu[j]).reshape(2,1)
            probs[j] =math.exp(-(xi - mu[j]).T.dot(inv_sigmas[j]).dot(xi - mu[j]) / 2)
            probs[j] /= (2 * math.pi * math.sqrt(det_sigmas[j]))

        #probs = probs / np.sum(probs)
        return -np.log(probs)
    
        
    def alpha_(self):
        alphas=[[] for t in range(self.T)]
        alphas[0]=self.pi
        print(self.pi,'pi')
        for index in range(1,self.T):
            probs=self.multivariate_gaussian(self.obs[index])
            result=np.zeros(4)
            for q in self.state:
                print('probq',probs[q])
                print('transition*alpha',np.sum(self.transition[q,:].dot(alphas[index-1])))
                result[q]=probs[q]*np.sum(self.transition[q,:].dot(alphas[index-1]))
                print('resul',result[q])
            alphas[index]=result
        return alphas 

#     def alpha_(self,index,probs): #backward
#         result=np.zeros(4)
#         if index==0:
#             return self.pi
#         else:
#             probs=self.multivariate_gaussian(self.obs[index-1])
#             alpha=self.alpha_(index-1,probs)
#             for q in self.state:
#                 result[q]= probs[q]*np.sum(self.transition[q,:].dot(alpha))
#             return result
        
    def beta_(self): #forwoard
        betas=[[] for t in range(self.T)]
        betas[(T-1)]=1
        for index in range((self.T-2),-1,-1):
            probs=self.multivariate_gaussian(self.obs[index])
            result=np.zeros(4)
            for q in self.state:
                result[q]=np.sum(probs[q]*self.transition[:,q].dot(betas[index+1]))
            betas[index]=result
        return betas
                
#         result=np.zeros(4)
#         if index == T:
#             return 1
#         else: 
#             beta = self.beta_(index+1,probs)
#             for q in self.state:
#                 result[q]=np.sum(probs[q]*self.transition[:,q].dot(beta))  
#             return result 
        
    def estimate_transition(self,alphas,betas):
        for k in range(4):
            for l in range(4):
                res=0
                denominateur=0
                for index in range(self.T):
                    probs=self.multivariate_gaussian(self.obs[index])
                    res+=self.transition[k,l]*alphas[index][l]*betas[index][k]*probs[k]
                    denominateur+=np.sum(alphas[index].dot(betas[index]))
                self.transition[k,l]=res/denominateur
        print('transision',self.transition)
        return self.transition
    
    def estimate_mu(self,alphas,betas):
        for k in range(4):
            res=0
            for index in range(self.T):
                res = res + alphas[index][k]*betas[index][k]*(self.obs[index])/np.sum([alphas[index][k]*betas[index][k] for index in range(T)])
            self.mu[k]=res
        print('selfmu',self.mu)
        return self.mu
    def estimate_sigma(self,alphas,betas):                         
        for k in range(4):
            res=0
            for index in range(self.T):
                x_u=self.obs[index]-self.mu[k]
                x_u=x_u.reshape(2,1)
                res = res+ alphas[index][k]*betas[index][k]*(x_u).dot(x_u.T)/np.sum([alphas[index][k]*betas[index][k] for index in range(T)])
            self.sigma[k]=res
        print('selfsigmas',self.sigma)
        return self.sigma
    
    def estimate_multinomial(self,alphas,betas):
        for k in range(4):
            self.pi[k]=alphas[0][k]*betas[0][k]/np.sum(alphas[0].dot(betas[0]))
        return self.pi
    
        
    def fit(self): 
        for j in range(10):
            #forward,backward (E-step)
            alphas=self.alpha_()
            betas=self.beta_()
            for index in range(self.T):
                self.q[index]=np.argmax(alphas[index] * betas[index] /np.sum(alphas[index].dot(betas[index])))
        
            #maximization(M-step)
            self.transition=self.estimate_transition(alphas,betas)
            self.mu=self.estimate_mu(alphas,betas)
            self.sigma=self.estimate_sigma(alphas,betas)
            self.multinomial=self.estimate_multinomial(alphas,betas)
            
            print('fin')
          
        return self.q
            
            
        
    

In [68]:
import pandas as pd

def read_file(filename):
    """ Load the data
    return:: x array, y array """
    print("Loading data " + filename)
    df = pd.read_csv(filename, delimiter=' ',  header=None)
    return df.values

filename = "EMGaussian.data"
filename_test = "EMGaussian.test"
x = read_file(filename)
x_test = read_file(filename_test)


Loading data EMGaussian.data
Loading data EMGaussian.test


In [69]:
from emgaussian import EMGaussian
from kmean import KMeans
import matplotlib.pyplot as plt
import math
#Initialize parameters with EMGaussian
emg = EMGaussian(x, isotropic=False)
emg.fit(plot_likelihood=False)

pi=emg.pi
mu=emg.mu
sigma=emg.sigmas
transition = np.array([ [0.25, 0.25, 0.25, 0.25], [0.25, 0.25, 0.25, 0.25], [0.25, 0.25, 0.25, 0.25], [0.25, 0.25, 0.25, 0.25]])
obs = x
T=x.shape[0]

#HMM
model = HMM(transition, mu,sigma,obs,T,pi)
model.fit()

Kmeans converged after 8 iterations, loss: 3237.668427
Fit Gaussian converged after 10 iterations
[0.29490093 0.24174905 0.24883438 0.21451564] pi
probq 150.4123951574003
transition*alpha 0.2499999999999999
resul 37.603098789350064
probq 25.527960210022414
transition*alpha 0.2499999999999999
resul 6.381990052505601
probq 4.055068755579151
transition*alpha 0.2499999999999999
resul 1.0137671888947872
probq 82.1362683138221
transition*alpha 0.2499999999999999
resul 20.534067078455514
probq 42.26904377606652
transition*alpha 16.383230777301492
resul 692.503498919157
probq 17.800118210181658
transition*alpha 16.383230777301492
resul 291.6234445006529
probq 13.44276775470623
transition*alpha 16.383230777301492
resul 220.23596641101918
probq 3.3404905001594303
transition*alpha 16.383230777301492
resul 54.72802677349523
probq 3.770758133929949
transition*alpha 314.77273415108107
resul 1186.9318476395583
probq 10.580996639273659
transition*alpha 314.77273415108107
resul 3330.6092421875696
probq

C:\Users\Hind Dadoun\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: RuntimeWarning: overflow encountered in double_scalars


 3.8552666125341055
transition*alpha inf
resul inf
probq 22.699535549837996
transition*alpha inf
resul inf
probq 9.545928568254572
transition*alpha inf
resul inf
probq 117.01482055513408
transition*alpha inf
resul inf
probq 26.51264211762139
transition*alpha inf
resul inf
probq 3.1240667433507645
transition*alpha inf
resul inf
probq 98.69930679254722
transition*alpha inf
resul inf
probq 12.96812162639174
transition*alpha inf
resul inf
probq 9.65042191273451
transition*alpha inf
resul inf
probq 9.41861537637146
transition*alpha inf
resul inf
probq 2.215797495485524
transition*alpha inf
resul inf
probq 98.65530742114692
transition*alpha inf
resul inf
probq 23.16836654992105
transition*alpha inf
resul inf
probq 2.9346328325722526
transition*alpha inf
resul inf
probq 83.81650352329774
transition*alpha inf
resul inf
probq 7.3402079502687645
transition*alpha inf
resul inf
probq 6.72537819754315
transition*alpha inf
resul inf
probq 8.627355536972065
transition*alpha inf
resul inf
probq 2.8551

C:\Users\Hind Dadoun\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarning: overflow encountered in double_scalars
C:\Users\Hind Dadoun\Anaconda3\lib\site-packages\ipykernel_launcher.py:121: RuntimeWarning: invalid value encountered in true_divide


TypeError: 'int' object is not subscriptable

In [7]:
T

500

In [11]:
range(10,0)

range(10, 0)

In [13]:
for i in range(10,0,-1):
    print(i)

10
9
8
7
6
5
4
3
2
1
